In [ ]:
#import Libraries
import pyspark
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#import pyspark in order to use sql
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql import types as T
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark import SparkContext

#import transformers and Berk tokenizer in order to transform twitter text to tokens
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)


#use this module to initialize the sparkContext
#Sparconf = pyspark.SparkConf()
conf = pyspark.SparkConf()
conf.setMaster("local").setAppName("test").set("spark.local.dir", "tmp/spark")
sc = pyspark.SparkContext(conf=conf)

#sc = SparkContext("local", "App Name")
spark = SQLContext(sc)


In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Row
from pyspark.sql.window import Window


conf = pyspark.SparkConf()
conf.setMaster("local").setAppName("test").set("spark.local.dir", "tmp/spark")
SparkContext.setSystemProperty('spark.driver.memory','16g')
SparkContext.setSystemProperty('spark.executor.memory','16g')
sc = pyspark.SparkContext(conf=conf)
#SparkConf conf = new SparkConf().setMaster("local”).setAppName("test”).set("spark.local.dir", "/tmp/spark-temp");

#sc = SparkContext("local", "App Name")
spark = SQLContext(sc)

In [2]:
@F.udf("String")
def decode_tokens(tokens):
  return tokenizer.decode(tokens)

In [4]:
#read training_parquet 
read_training_parquet ="parquet_files/training_df.parquet"
training_df = spark.read.parquet(read_training_parquet)
training_df.count()

667677519

In [3]:
#read validation_dataset
read_validation_parquet = "parquet_files/validation_df.parquet"
validation_df =spark.read.parquet(read_validation_parquet)
#validation_df.count()

28542288

In [5]:
#unique tweets

training_tweets = training_df.dropDuplicates(["tweet_id"])\
.select("tweet_id",
        decode_tokens(F.col("text_tokens")).alias("text"),
        "tweet_type",
        "language",
        F.hour(F.to_timestamp("timestamp")).alias("hour_tweet"),
        F.size("text_tokens").alias("num_tokens"),
        F.when(F.col("hashtags").isNull(), 0).otherwise(1).alias("has_hashtags"),
        F.when(F.col("present_media").isNull(), 0).otherwise(1).alias("has_media"), 
        F.when(F.col("present_links").isNull(), 0).otherwise(1).alias("has_links"))
training_tweets.count()

261029636

In [ ]:
tweet_in_top_hashtag_daily = training_df\
.dropDuplicates(["tweet_id"])\
.select("tweet_id", "timestamp", F.explode("hashtags").alias("hashtags_exploded"))\
.withColumn("hashtag_duration", F.max("timestamp").over(Window.partitionBy("hashtags_exploded")))\
.withColumn("hashtag_duration", F.col("hashtag_duration") - F.min("timestamp").over(Window.partitionBy("hashtags_exploded")))\
.withColumn("hashtag_duration2", F.col("hashtag_duration") / (24*3600))\
.withColumn("hashtag_duration3", F.round("hashtag_duration2"))\
.withColumn("is_in_top_daily_hashtags", F.when( ((F.col("hashtag_duration3") > 0) & (F.abs(F.col("hashtag_duration3") - F.col("hashtag_duration2")) < 0.007)), 1).otherwise(0))\
.select("tweet_id", "is_in_top_daily_hashtags")\
.groupBy("tweet_id").agg(F.max("is_in_top_daily_hashtags").alias("is_in_top_daily_hashtags"))


In [ ]:
#concatenate matrices
training_tweets = training_tweets.join(tweet_in_top_hashtag_daily, "tweet_id", "left_outer")\
.withColumn("is_in_top_daily_hashtags", F.when(F.col("is_in_top_daily_hashtags").isNull(), 0).otherwise(F.col("is_in_top_daily_hashtags")))

In [ ]:
tweet_in_top_link_daily =  training_df\
.dropDuplicates(["tweet_id"])\
.select("tweet_id", "timestamp", F.explode("present_links").alias("present_links_exploded"))\
.withColumn("link_duration", F.max("timestamp").over(Window.partitionBy("present_links_exploded")))\
.withColumn("link_duration", F.col("link_duration") - F.min("timestamp").over(Window.partitionBy("present_links_exploded")))\
.withColumn("link_duration2", F.col("link_duration") / (24*3600))\
.withColumn("link_duration3", F.round("link_duration2"))\
.withColumn("is_in_top_daily_links", F.when( ((F.col("link_duration3") > 0) & (F.abs(F.col("link_duration3") - F.col("link_duration2")) < 0.007)), 1).otherwise(0))\
.select("tweet_id", "is_in_top_daily_links")\
.groupBy("tweet_id").agg(F.max("is_in_top_daily_links").alias("is_in_top_daily_links"))

In [ ]:
#concatenate matrices
training_tweets = training_tweets.join(tweet_in_top_link_daily, "tweet_id", "left_outer")\
.withColumn("is_in_top_daily_links", F.when(F.col("is_in_top_daily_links").isNull(), 0).otherwise(F.col("is_in_top_daily_links")))

In [ ]:
training_tweets_parquet = "parquet_files/training_tweets.parquet"
training_tweets.write.parquet(training_tweets_parquet)
#training_tweets.write.parquet("/media/intelligence-lab-pc3/1.0 TB Hard Disk/parquet_files/training_tweets.parquet")

In [ ]:
#import libraries for Pipeline and TF-IDF
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, HashingTF, IDF, CountVectorizer

In [ ]:
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="(?U)\\bhttps?://\\S*|#?\\b\\w+\\b", gaps=False)
languages = ["english", "french", "spanish", "german", "finnish", "turkish", "english", "russian", "norwegian", "dutch", "danish", "hungarian", "italian", "swedish", "portuguese"]
stopWords = []

for i in languages:
    stopWords += StopWordsRemover.loadDefaultStopWords(i)
removeOtherWords = ["@", "sep","unk", "cls","rt"]

stopWords += removeOtherWords

stopWordsRemover = StopWordsRemover(inputCol=regexTokenizer.getOutputCol(), outputCol="tokens2", stopWords=stopWords)

hashingTF = HashingTF(inputCol=stopWordsRemover.getOutputCol(), outputCol="hashedTF", numFeatures=16)

IDF = IDF(inputCol=hashingTF.getOutputCol(), outputCol="text_features")

stages = [regexTokenizer, stopWordsRemover, hashingTF, IDF]

tf_idf_pipeline = Pipeline(stages=stages)


In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, FeatureHasher
from pyspark.ml import Pipeline

categoricalColumns = ["tweet_type", "language", "hour_tweet", "has_hashtags", "has_media", "has_links", "is_in_top_daily_hashtags", "is_in_top_daily_links"]

stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    stages += [stringIndexer]
    
featInputs = [c + "Index" for c in categoricalColumns]
featHasher = FeatureHasher(numFeatures=16, inputCols=featInputs, outputCol="otherFeaturesHashed", categoricalCols=featInputs)
stages += [featHasher]

other_tweet_features_pipline = Pipeline(stages=stages)    

In [ ]:
assemble_tweet_features = VectorAssembler(inputCols=["text_features", "otherFeaturesHashed"], outputCol="tweet_features")


In [ ]:
create_tweet_features = Pipeline(stages=[tf_idf_pipeline, other_tweet_features_pipline, assemble_tweet_features]).fit(training_tweets)

In [ ]:
create_tweet_features.transform(training_tweets).select("tweet_id", "tweet_features").show()

In [ ]:
create_tweet_features_model = "/media/intelligence-lab-pc3/1.0 TB Hard Disk/jupyter_notebook/models/create_tweets_features.model"
create_tweet_features.save(create_tweet_features_model)

In [ ]:
from pyspark.ml import PipelineModel
create_tweet_features = PipelineModel.load(create_tweet_features_model)
#create_tweet_features.transform(training_tweets).select("tweet_id", "tweet_features").show()

In [6]:
validation_tweets = validation_df.dropDuplicates(["tweet_id"])\
.select("tweet_id",
        decode_tokens(F.col("text_tokens")).alias("text"),
        "tweet_type",
        "language",
        F.hour(F.to_timestamp("timestamp")).alias("hour_tweet"),
        F.size("text_tokens").alias("num_tokens"),
        F.when(F.col("hashtags").isNull(), 0).otherwise(1).alias("has_hashtags"),
        F.when(F.col("present_media").isNull(), 0).otherwise(1).alias("has_media"),
        F.when(F.col("present_links").isNull(), 0).otherwise(1).alias("has_links"))

In [7]:
validation_tweets.count()

10134368

In [4]:
validation_tweets = validation_df.dropDuplicates(["tweet_id"])\
.select("tweet_id",
        decode_tokens(F.col("text_tokens")).alias("text"),
        "tweet_type",
        "language",
        F.hour(F.to_timestamp("timestamp")).alias("hour_tweet"),
        F.size("text_tokens").alias("num_tokens"),
        F.when(F.col("hashtags").isNull(), 0).otherwise(1).alias("has_hashtags"),
        F.when(F.col("present_media").isNull(), 0).otherwise(1).alias("has_media"), 
        F.when(F.col("present_links").isNull(), 0).otherwise(1).alias("has_links"))
        
tweet_is_in_top_daily_hashtag = validation_df\
.dropDuplicates(["tweet_id"])\
.select("tweet_id", "timestamp", F.explode("hashtags").alias("hashtags_exploded"))\
.withColumn("hashtag_duration", F.max("timestamp").over(Window.partitionBy("hashtags_exploded")))\
.withColumn("hashtag_duration", F.col("hashtag_duration") - F.min("timestamp").over(Window.partitionBy("hashtags_exploded")))\
.withColumn("hashtag_duration2", F.col("hashtag_duration") / (24*3600))\
.withColumn("hashtag_duration3", F.round("hashtag_duration2"))\
.withColumn("is_in_top_daily_hashtags", F.when( ((F.col("hashtag_duration3") > 0) & (F.abs(F.col("hashtag_duration3") - F.col("hashtag_duration2")) < 0.007)), 1).otherwise(0))\
.select("tweet_id", "is_in_top_daily_hashtags")\
.groupBy("tweet_id").agg(F.max("is_in_top_daily_hashtags").alias("is_in_top_daily_hashtags"))

validation_tweets = validation_tweets.join(tweet_is_in_top_daily_hashtag, "tweet_id", "left_outer")\
.withColumn("is_in_top_daily_hashtags", F.when(F.col("is_in_top_daily_hashtags").isNull(), 0).otherwise(F.col("is_in_top_daily_hashtags")))

tweet_is_in_top_daily_link = validation_df\
.dropDuplicates(["tweet_id"])\
.select("tweet_id", "timestamp", F.explode("present_links").alias("present_links_exploded"))\
.withColumn("link_duration", F.max("timestamp").over(Window.partitionBy("present_links_exploded")))\
.withColumn("link_duration", F.col("link_duration") - F.min("timestamp").over(Window.partitionBy("present_links_exploded")))\
.withColumn("link_duration2", F.col("link_duration") / (24*3600))\
.withColumn("link_duration3", F.round("link_duration2"))\
.withColumn("is_in_top_daily_links", F.when( ((F.col("link_duration3") > 0) & (F.abs(F.col("link_duration3") - F.col("link_duration2")) < 0.007)), 1).otherwise(0))\
.select("tweet_id", "is_in_top_daily_links")\
.groupBy("tweet_id").agg(F.max("is_in_top_daily_links").alias("is_in_top_daily_links"))

validation_tweets = validation_tweets.join(tweet_is_in_top_daily_link, "tweet_id", "left_outer")\
.withColumn("is_in_top_daily_links", F.when(F.col("is_in_top_daily_links").isNull(), 0).otherwise(F.col("is_in_top_daily_links")))

#validation_tweets.write.parquet("/media/user2/TOSHIBA EXT/code/parquet_files/validation_tweets.df")

In [5]:
#validation_tweets.write.parquet("/media/user2/TOSHIBA EXT/code/parquet_files/validation_tweets.df")
validation_tweets.write.parquet("/media/user2/2.0 TB Hard Disk/parquet_files/validation_tweets.df")

Py4JJavaError: An error occurred while calling o218.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apac